## Automated Evaluation of AI-Generated Loan Decision Explanations Using Simulated Expert Assessment

This notebook implements an automated evaluation framework that uses GPT-4o to simulate both Credit Risk Professional (CRP) and Non-Credit Risk Professional (NCRP) personas for rating AI-generated loan decision explanations across 8 key metrics: Understandability, Trustworthiness, Insightfulness, Satisfaction, Confidence, Convincingness, Communicability, and Usability.

Note: This script is intended for academic reference only.

In [ ]:
import numpy as np

import scipy.stats as stats

import openai
from tenacity import retry, stop_after_attempt, wait_exponential

from dotenv import load_dotenv

import os
import json

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

METRICS = {
    "Understandability": {
        "crp": "The explanation is easy to understand.",
        "ncrp": "The explanation is easy to understand."
    },
    "Trustworthiness": {
        "crp": "The explanation can be trusted because it presents sufficient and reliable evidence to support the loan decision.",
        "ncrp": "I trust this explanation given by the model."
    },
    "Insightfulness": {
        "crp": "This explanation reveals insightful risk factors that influence loan decisions.",
        "ncrp": "This explanation provides useful insight into why the decision was made."
    },
    "Satisfaction": {
        "crp": "The level of details satisfies my expectations for a risk assessment.",
        "ncrp": "I am satisfied with how the explanation addressed my concerns about the decision."
    },
    "Confidence": {
        "crp": "This explanation increases my confidence for loan approval/denial.",
        "ncrp": "I am confident in the explanation provided for the model's decision."
    },
    "Convincingness": {
        "crp": "The justification is convincing and argues for the decision using well-weighted risk evidence.",
        "ncrp": "The explanation is convincing and makes the decision seem reasonable."
    },
    "Communicability": {
        "crp": "I could use this explanation to communicate with customers and other teams in the company.",
        "ncrp": "The explanation is clear enough for me to communicate with others."
    },
    "Usability": {
        "crp": "This explanation could be directly used in loan approval/denial.",
        "ncrp": "I am likely to use this explanation in the future because it supports my decision-making."
    }
}

ACRONYMS = {
    "Understandability": "UND",
    "Trustworthiness": "TRU", 
    "Insightfulness": "INS",
    "Satisfaction": "SAT",
    "Confidence": "CON",
    "Convincingness": "CVN",
    "Communicability": "COM",
    "Usability": "USB"
}

@retry(stop=stop_after_attempt(5), wait=wait_exponential(min=3, max=30))
def get_likert_scores(explanation, persona="crp"):
    
    criteria = "\n".join([f"{i+1}. {METRICS[m][persona]}" for i, m in enumerate(METRICS)])
    scale_definition = (
        "**Use the following scale for all ratings:**\n"
        "- 1 = Strongly Disagree\n"
        "- 2 = Disagree\n"
        "- 3 = Neutral\n"
        "- 4 = Agree\n"
        "- 5 = Strongly Agree"
    )
    
    if persona == "crp":
        role_description = (
            "You are a Credit Risk Professional evaluating AI-generated explanations. "
            "Assess each explanation as if you were reviewing it in a formal credit-risk setting. "
            "Consider evidential support, decision justification, and operational relevance."
        )
    else:
        role_description = (
            "You are a general Non-Credit Risk Professional evaluating AI-generated explanations. "
            "Assess the explanation from a general reader's perspective, focusing on clarity, "
            "perceived insight, and general usefulness without requiring credit-specific expertise."
        )
    
    input_text = f"""{role_description}

Rate the following explanation based on these 8 criteria:
{criteria}
{scale_definition}

Return a JSON dictionary with numeric keys ("1", "2", "3", etc.) mapping to scores from 1 to 5 (integers only). 
Do not include commentary, explanation, or extra formatting—return only the JSON object.

Explanation:
\"\"\"{explanation}\"\"\""""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[system_msg, user_msg],
            temperature=0.2,
            max_tokens=1000,
            response_format={"type": "json_object"}
        )

        content = response.choices[0].message.content
        
        raw_scores = json.loads(content)
        
        metric_names = list(METRICS.keys())
        converted_scores = {}
        
        for i, metric_name in enumerate(metric_names):
            key = str(i + 1) 
            if key in raw_scores:
                converted_scores[metric_name] = raw_scores[key]
        
        return converted_scores
        
    except Exception as e:
        print(f"Error in API call: {e}")
        return {} 

def compute_ci(scores):

    mean = np.mean(scores)
    stderr = stats.sem(scores)
    margin = stderr * stats.t.ppf((1 + 0.95) / 2, len(scores) - 1)
    ci_low = max(1.0, mean - margin)
    ci_high = min(5.0, mean + margin)
    return mean, ci_low, ci_high

def print_round_scores(round_num, crp_scores, ncrp_scores):
    
    print(f"\n### ROUND {round_num} ###\n")
    
    print("CRP:", end=" ")
    crp_values = []
    for metric in METRICS:
        acronym = ACRONYMS[metric]
        score = crp_scores.get(metric, 0)
        crp_values.append(f"{acronym}:{score}")
    print(" ".join(crp_values))
    
    print("NCRP:", end=" ")
    ncrp_values = []
    for metric in METRICS:
        acronym = ACRONYMS[metric]
        score = ncrp_scores.get(metric, 0)
        ncrp_values.append(f"{acronym}:{score}")
    print(" ".join(ncrp_values))

def print_final_summary(all_crp_scores, all_ncrp_scores):

    print("### Final Summary with 95% Confidence Intervals ###")
    
    print("\nCRP Summary:")
    for metric in METRICS:
        acronym = ACRONYMS[metric]
        scores = all_crp_scores[metric]
        if len(scores) >= 2:
            mean, ci_low, ci_high = compute_ci(scores)
            margin = ci_high - mean
            print(f"{acronym}: {mean:.2f} ± {margin:.2f}")
        else:
            print(f"{acronym}: N/A")
    
    print("\nNCRP Summary:")
    for metric in METRICS:
        acronym = ACRONYMS[metric]
        scores = all_ncrp_scores[metric]
        if len(scores) >= 2:
            mean, ci_low, ci_high = compute_ci(scores)
            margin = ci_high - mean
            print(f"{acronym}: {mean:.2f} ± {margin:.2f}")
        else:
            print(f"{acronym}: N/A")

def run_eval(path, max_samples=50):

    with open(path, "r") as f:
        lines = [json.loads(line) for line in f][:max_samples]
    
    print(f"Running ChatGPT Simulated Ratings on {len(lines)} samples\n")
    
    all_crp_scores = {metric: [] for metric in METRICS}
    all_ncrp_scores = {metric: [] for metric in METRICS}
    
    for i, example in enumerate(lines, 1):
        print(f"\nProcessing explanation {i}/{len(lines)}...")
        
        crp_scores = get_likert_scores(example["explanation"], "crp")
        print("CRP scores retrieved successfully")
            
        ncrp_scores = get_likert_scores(example["explanation"], "ncrp")
        print("NCRP scores retrieved successfully")
        
        for metric in METRICS:
            if metric in crp_scores:
                all_crp_scores[metric].append(crp_scores[metric])
            if metric in ncrp_scores:
                all_ncrp_scores[metric].append(ncrp_scores[metric])
        
        print_round_scores(i, crp_scores, ncrp_scores)
    
    print_final_summary(all_crp_scores, all_ncrp_scores)
    
    return all_crp_scores, all_ncrp_scores


# print("Evaluating Gemini-XGB")
# print("Evaluating Gemini-GAT")
# print("Evaluating Gemini-Hybrid")
# print("Evaluating Gemma3-XGB")
# print("Evaluating Gemma3-GAT")
# print("Evaluating Gemma3-Hybrid")
# print("Evaluating DeepSeek-R1-XGB")
# print("Evaluating DeepSeek-R1-GAT")
print("Evaluating DeepSeek-R1-Hybrid")

# EXPLANATION_PATH = "../models/explanations/gemini_xgb_explanations.jsonl"
# EXPLANATION_PATH = "../models/explanations/gemini_gnn_explanations.jsonl"
# EXPLANATION_PATH = "../models/explanations/gemini_hybrid_explanations.jsonl"
# EXPLANATION_PATH = "../models/explanations/gemma_xgb_explanations.jsonl"
# EXPLANATION_PATH = "../models/explanations/gemma_gnn_explanations.jsonl"
# EXPLANATION_PATH = "../models/explanations/gemma_hybrid_explanations.jsonl"
# EXPLANATION_PATH = "../models/explanations/deepseek_xgb_explanations.jsonl"
# EXPLANATION_PATH = "../models/explanations/deepseek_gnn_explanations.jsonl"
EXPLANATION_PATH = "../models/explanations/deepseek_hybrid_explanations.jsonl"

crp_scores, ncrp_scores = run_eval(EXPLANATION_PATH, max_samples=100) 